#Term Project - Inventory Tracking

In [139]:
import requests
import ipywidgets as widgets
from IPython.display import display, HTML
import functools

In [140]:
from bqplot import pyplot as plt
import bqplot
import numpy as np
import pandas as pd

In [141]:
local_url = "http://127.0.0.1:5000/los-cangri/"
online_url = "https://los-cangris-d8db459397d0.herokuapp.com/los-cangri/"
local = True

base_url = local_url if local else online_url 

#helper url's
warehouse_url = base_url+'warehouse'
most_url = base_url+'most/'
least_url = base_url+'least/'
part_url = base_url+'part'
supplier_url = base_url+'supplier/'

In [142]:
data = requests.get(warehouse_url)
# data

In [143]:
from markdown import markdown
accordion = widgets.Accordion()
text_global_stats = """
### global statistics
a. /most/rack – Top 10 warehouses with the most racks

b. /most/incoming – Top 5 warehouses with the most incoming transactions

c. /most/deliver – Top 5 warehouses that delivers the most exchanges

d. /most/transactions – Top 3 users that made the most transactionse.

e. /least/outgoing – Top 3 warehouses with the least outgoing transactions

f. /most/city – Top 3 warehouses’ cities with the most transactions
"""

text_local_stats = """
### local statistics

a. /warehouse/<id>/profit – Warehouse’s year profit

b. /warehouse/<id>/rack/lowstock – Top 5 racks with quantity under the 25% capacity 
threshold

c. /warehouse/<id>/rack/material – Bottom 3 part’s type/material in the warehouse

d. /warehouse/<id>/rack/expensive – Top 5 most expensive racks in the warehouse

e. /warehouse/<id>/transaction/suppliers – Top 3 supplier that supplied to the warehouse 

f. /warehouse/<id>/transaction/leastcost – Top 3 days with the smallest incoming 
transactions’ cost

g. /warehouse/<id>/users/receivesmost – Top 3 users that receives the most exchanges
"""

text_global_stats = widgets.HTML(value=markdown(text_global_stats))
text_local_stats = widgets.HTML(value=markdown(text_local_stats))

accordion.children = [text_global_stats, text_local_stats]
accordion.set_title(0, 'Global Statistics Endpoints')
accordion.set_title(1, 'Local Statistics Endpoints')
display(accordion)

Accordion(children=(HTML(value='<h3>global statistics</h3>\n<p>a. /most/rack – Top 10 warehouses with the most…

## Helper Functions

<!-- ### global statistics
a. /most/rack – Top 10 warehouses with the most racks

b. /most/incoming – Top 5 warehouses with the most incoming transactions

c. /most/deliver – Top 5 warehouses that delivers the most exchanges

d. /most/transactions – Top 3 users that made the most transactionse.

e. /least/outgoing – Top 3 warehouses with the least outgoing transactions

f. /most/city – Top 3 warehouses’ cities with the most transactions -->


In [144]:
def get_most_racks():
    data = requests.get(most_url+'rack')
    return data.json()

def get_most_incoming():
    data = requests.get(most_url+'incoming')
    return data.json()

def get_most_deliver():
    data = requests.get(most_url+'deliver')
    return data.json()  

def get_most_transactions():
    data = requests.get(most_url+'transactions') # most transactions endpoint does not exist - need to FIX
    return data.json()  

def get_least_outgoing():
    data = requests.get(least_url+'outgoing')
    return data.json()  

def get_most_city():
    data = requests.get(most_url+'city')
    return data.json()  



In [145]:
def display_warehouses_global_stat_graph(json, label):
    # count vs wid
    racks = json['Warehouses']
    wids = [pair['wid'] for pair in racks]
    counts = [pair['count'] for pair in racks]
    fig = plt.figure()
    bar_chart = plt.bar(
        wids, 
        counts, 
        scales={'x':bqplot.OrdinalScale()}
    )
    plt.xlabel('WID')
    plt.ylabel('Count')
    plt.title(label)

    plt.show()

In [146]:
def display_warehouse_with_most_racks():
    label = "Warehouses with most racks"
    json = get_most_racks()
    display_warehouses_global_stat_graph(json, label)

# display_warehouse_with_most_racks()

In [147]:
def display_warehouse_with_most_incoming_transactions():
    label = "Warehouses with most incoming transactions"
    json = get_most_incoming()
    display_warehouses_global_stat_graph(json, label)

# display_warehouse_with_most_incoming_transactions()

In [148]:
def display_warehouse_with_most_exchanges():
    label = "Warehouses with most exchanges"
    json = get_most_deliver()
    display_warehouses_global_stat_graph(json, label)

# display_warehouse_with_most_exchanges()

In [149]:
def display_warehouse_with_least_outgoing():
    label = "Warehouses with least outgoing"
    json = get_least_outgoing()
    display_warehouses_global_stat_graph(json, label)

# display_warehouse_with_least_outgoing()

In [150]:
def display_cities_with_most_warehouse_transactions():
    label = "Cities with most warehouse transactions"
    json = get_most_city()
    cities_pair = json['Citites']
    cities = [pair['city'] for pair in cities_pair]
    counts = [pair['count'] for pair in cities_pair]
    fig = plt.figure()
    bar_chart = plt.bar(
        cities, 
        counts, 
        scales={'x':bqplot.OrdinalScale()}
    )
    plt.xlabel('Cities')
    plt.ylabel('Count')
    plt.title(label)

    plt.show()

# display_cities_with_most_warehouse_transactions()

In [151]:
def display_top_three_users_with_most_transactions():
    label = "Top 3 users with most transactions"
    json = get_most_transactions()
    users_pair = json['Users']
    cities = [pair['uid'] for pair in users_pair]
    counts = [pair['count'] for pair in users_pair]
    fig = plt.figure()
    bar_chart = plt.bar(
        cities, 
        counts, 
        scales={'x':bqplot.OrdinalScale()}
    )
    plt.xlabel('Users')
    plt.ylabel('Count')
    plt.title(label)

    plt.show()

# display_top_three_users_with_most_transactions()

<!-- ### Local Statistics

a. /warehouse/<id>/profit – Warehouse’s year profit

b. /warehouse/<id>/rack/lowstock – Top 5 racks with quantity under the 25% capacity 
threshold

c. /warehouse/<id>/rack/material – Bottom 3 part’s type/material in the warehouse

d. /warehouse/<id>/rack/expensive – Top 5 most expensive racks in the warehouse

e. /warehouse/<id>/transaction/suppliers – Top 3 supplier that supplied to the warehouse 

f. /warehouse/<id>/transaction/leastcost – Top 3 days with the smallest incoming 
transactions’ cost

g. /warehouse/<id>/users/receivesmost – Top 3 users that receives the most exchanges -->

In [152]:
def get_warehouse_profit(wid:int, uid:int):
    perms = {'User_id': uid}
    data = requests.post(warehouse_url+f'/{wid}/profit', json=perms)
    return data.json()

def get_warehouse_lowstock(wid:int, uid:int):
    perms = {'User_id': uid}
    data = requests.post(warehouse_url+f'/{wid}/rack/lowstock', json=perms)        
    return data.json()

def get_warehouse_expensive(wid:int, uid:int):
    perms = {'User_id': uid}
    data = requests.post(warehouse_url+f'/{wid}/rack/expensive', json=perms)        
    return data.json()

def get_warehouse_bottom_material(wid:int, uid:int):
    perms = {'User_id': uid}
    data = requests.post(warehouse_url+f'/{wid}/rack/material', json=perms)        
    return data.json()

def get_warehouse_suppliers(wid:int, uid:int):
    perms = {'User_id': uid}
    data = requests.post(warehouse_url+f'/{wid}/transaction/suppliers', json=perms)        
    return data.json()

def get_warehouse_receivesmost(wid:int, uid:int):
    perms = {'User_id': uid}
    data = requests.post(warehouse_url+f'/{wid}/users/receivemost', json=perms)        
    return data.json()

def get_warehouse_leastcost(wid:int, uid:int): #this one btw
    perms = {'User_id': uid}
    data = requests.post(warehouse_url+f'/{wid}/transaction/leastcost', json=perms)        
    return data.json()

def get_warehouse_data(wid:int):
    data = requests.get(warehouse_url+f'/{wid}')
    return data.json()

def get_part_data():
    data = requests.get(part_url)
    return data.json() 

def get_supplier_data(sid:int):
    data = requests.get(supplier_url+f'{sid}')
    return data.json()

def get_parts_supplied(sid:int):
    data = requests.get(supplier_url+f'{sid}/parts')
    return data.json()

def get_parts_in_warehouse(wid:int, uid:int):
    perms = {'User_id': uid}
    data = requests.post(warehouse_url+f'/parts/{wid}', json=perms)
    return data.json()
#print(get_part_data())
#print(get_supplier_data(2))
#print(get_parts_in_warehouse(10, 24))
#print(get_warehouse_data(10))
#print(get_warehouse_leastcost(7,10))
#print(get_warehouse_receivesmost(1, 2))

In [153]:
def display_warehouse_profit(out, wid, uid):
    json_data = get_warehouse_profit(wid, uid)
    # Ensure 'Profit' key is present in the JSON
    if 'Profits' not in json_data or not json_data['Profits']:
        print("Error: 'Profit' key not found in JSON.")
        return

    profit_data = json_data['Profits']
    x = [int(x['year']) for x in profit_data]
    y = [float(y['profit']) for y in profit_data]

    sorted_data = sorted(zip(x, y), key=lambda entry: entry[0])
    x_sorted, y_sorted = zip(*sorted_data)
        
    # Plot the profit data using a bar chart
    fig = plt.figure()
    bar_chart = plt.bar(x_sorted,y_sorted, scales={'x': bqplot.OrdinalScale()}) 
    
    plt.xlabel('Year')
    plt.ylabel('Profit')
    plt.title(f'Profits by year for Warehouse: {wid}')

    # Show the plot    
    with out:
        plt.show()

def display_warehouse_lowstock(out, wid, uid):
    json_data = get_warehouse_lowstock(wid, uid)
    
    # Check if 'Racks' key is present in the JSON
    if 'Racks' not in json_data:
        print("Error: 'Racks' key not found in JSON.")
        return

    racks_data = json_data['Racks']
    
    sorted_racks = sorted(racks_data, key=lambda x: x['quantity'], reverse=False)
    
    # Extract relevant information for plotting only the top 5 racks
    top_rack_labels = [rack['rid'] for rack in sorted_racks[:6]]
    top_quantities = [rack['quantity'] for rack in sorted_racks[:6]]

    # Plot the data using a bar chart
    fig = plt.figure()
    bar_chart = plt.bar(
        top_rack_labels,
        top_quantities,
        scales={'x': bqplot.OrdinalScale()},
    )
    
    plt.xlabel('Rack ID')
    plt.ylabel('Quantity')
    plt.title(f'Top 5 Racks with Low Stock for Warehouse {wid}')

    # Show the plot    
    with out:
        plt.show()
    
def display_warehouse_material(out, wid, uid):
    json_data = get_warehouse_bottom_material(wid, uid)
    
    # Check if 'Racks' key is present in the JSON
    if 'Racks' not in json_data:
        print("Error: 'Racks' key not found in JSON.")
        return

    racks_data = json_data['Racks']

    # Sort racks_data based on quantity in ascending order (bottom to top)
    sorted_racks = sorted(racks_data, key=lambda x: x['count'])
    
    # Extract relevant information for plotting only the bottom 3 material types
    bottom_material = [rack['ptype'] for rack in sorted_racks[:4]]
    bottom_quantities = [rack['count'] for rack in sorted_racks[:4]]

    # Plot the data using a bar chart
    fig = plt.figure()
    bar_chart = plt.bar(
        bottom_material,
        bottom_quantities,
        scales={'x': bqplot.OrdinalScale()},
    )
    
    plt.xlabel('Material')
    plt.ylabel('Quantity')
    plt.title(f'Bottom 3 Material Types for Warehouse: {wid}')

    # Show the plot    
    with out:
        plt.show()
    
def display_warehouse_expensive(out, wid, uid):
    json_data = get_warehouse_expensive(wid, uid)
    
    # Check for errors in the response
    if 'Racks' not in json_data:
        print(json_data)
        return
    
    racks_data = json_data['Racks']
    
    # Extract relevant information for plotting
    rack_labels = [rack['rid'] for rack in racks_data]
    total_prices = [rack['total_price'] for rack in racks_data]

    # Plot the data using a bar chart
    fig = plt.figure()
    bar_chart = plt.bar(
        rack_labels,
        total_prices,
        scales={'x': bqplot.OrdinalScale()},
    )
    
    plt.xlabel('Rack ID')
    plt.ylabel('Total Price')
    plt.title(f'Top 5 Most Expensive Racks for Warehouse: {wid}')

    # Show the plot    
    with out:
        plt.show()
        
    
def display_warehouse_suppliers(out, wid, uid):
    json_data = get_warehouse_suppliers(wid, uid)
    
    # Check if 'Suppliers' key is present in the JSON
    if 'Suppliers' not in json_data:
        print("Error: 'Suppliers' key not found in JSON.")
        return

    suppliers_data = json_data['Suppliers']

    # Sort suppliers_data based on quantity in ascending order (bottom to top)
    sorted_sups = sorted(suppliers_data, key=lambda x: x['count'], reverse=True)
    
    top_suppliers = [rack['sid'] for rack in sorted_sups[:4]]
    count = [rack['count'] for rack in sorted_sups[:4]]

    # Plot the data using a bar chart
    fig = plt.figure()
    bar_chart = plt.bar(
        top_suppliers,
        count,
        scales={'x': bqplot.OrdinalScale()},
    )
    
    plt.xlabel('Suppliers ID')
    plt.ylabel('Count')
    plt.title(f'Top 3 Suppliers that supplied to the Warehouse: {wid}')

    # Show the plot    
    with out:
        plt.show()
    
def display_warehouse_receivesmost(out, wid, uid):
    json_data = get_warehouse_receivesmost(wid, uid)
    
    # Check if 'Users' key is present in the JSON
    if 'Users' not in json_data:
        print("Error: 'Users' key not found in JSON.")
        return

    users_data = json_data['Users']

    # Sort user_data based on quantity in descending order 
    sorted_user = sorted(users_data, key=lambda x: x['count'], reverse=True)
    
    top_user = [rack['uid'] for rack in sorted_user[:4]]
    count = [rack['count'] for rack in sorted_user[:4]]

    # Plot the data using a bar chart
    fig = plt.figure()
    bar_chart = plt.bar(
        top_user,
        count,
        scales={'x': bqplot.OrdinalScale()},
    )
    
    plt.xlabel('User ID')
    plt.ylabel('Count')
    plt.title(f'Top 3 Users that received the most exchanges to Warehouse {wid}')

    # Show the plot    
    with out:
        plt.show()

def display_warehouse_leastcost(out, wid, uid):
    out.clear_output()
    json_data = get_warehouse_leastcost(wid, uid)
    if 'Dates' not in json_data:
        print("Error: 'Dates' key not found in JSON.")
        return
    date_data = json_data['Dates']
    # Sort user_data based on amount in descending order 
    sorted_dates = sorted(date_data, key=lambda x: x['amount'])
    amounts = [row['amount'] for row in sorted_dates[:4]]
    dates = [row['date'] for row in sorted_dates[:4]]


    fig = plt.figure()
    bar_chart = plt.bar(
        dates,
        amounts,
        scales={'x': bqplot.OrdinalScale()},
    )
    
    plt.xlabel('Dates')
    plt.ylabel('Amount')
    plt.title(f'Top 3 days with the smallest incoming transactions’ cost for Warehouse: {wid}')

    # Show the plot    
    with out:
        plt.show()

def display_warehouse_local_stat_graph(json, label):
    return NotImplemented

In [154]:
def display_part_prices(out, limit):
    out.clear_output()

    try:
        limit_val = limit.new
    except:
        limit_val = limit
    
    json = get_part_data()
    parts_info = json['Parts']
    if not limit_val or limit_val >= len(parts_info):
        max_display = len(parts_info)
    else:
        max_display = limit_val

    pids = [part_tup['pid'] for part_tup in parts_info][0:max_display]
    prices = [part_tup['pprice'] for part_tup in parts_info][0:max_display]
    
    label = 'Part Prices'
    fig = plt.figure()
    bar_chart = plt.bar(
        pids, 
        prices, 
        scales={'x':bqplot.OrdinalScale()}
    )
    plt.xlabel('PID')
    plt.ylabel('Price')
    plt.title(label)
    with out:
        plt.show()

    #try and except here to avoid weird errors
    
    # count = 0
    # with out:
    #     for parts_tup  in parts_info:
    #         if count >= max_display:
    #             break
    #         pname = parts_tup['pname']
    #         ptype = parts_tup['ptype']
    #         pprice = parts_tup['pprice']
    #         pid = parts_tup['pid']
    #         print(f'The "{ptype} {pname}" costs: ${pprice} (pid: {pid})')
    #         count += 1


In [155]:
def display_supplied_parts(out, sid):
    out.clear_output()
    supplier_id = None
    try:
        supplier_id = sid.new
    except:
        supplier_id = sid
    parts_json = get_parts_supplied(supplier_id)
    supplier_json = get_supplier_data(supplier_id)
    
    parts_info = parts_json['Parts']
    supplier_info = supplier_json['Supplier']
    sname = supplier_info['sname']
    sname = sname.strip() #eliminate any whitespace

    df = pd.DataFrame(data=parts_info)
    df = df.style.set_caption(f'"{sname}" supplies the following parts:')
    df = df.to_html(classes='center-table')
    with out:
        display(HTML(f'<div style="display: flex; justify-content: center;">{df}</div>'))

        # print(f'"{sname}" supplies the following parts:')
        # for parts_tup  in parts_info:
        #     pname = parts_tup['pname']
        #     ptype = parts_tup['ptype']
        #     pid = parts_tup['pid']
        #     print(f'"{ptype} {pname}" (pid: {pid})')
    


In [156]:
def display_parts_in_warehouse(out, warehouse, user):
    out.clear_output()
    wid = None
    uid = None
    try:
        wid = warehouse.new
        uid = user.new
    except:
        wid = warehouse
        uid = user

    json = get_parts_in_warehouse(wid, uid)  
    parts_info = None
    try:
        parts_info = json['Parts_in_warehouse']
    except:
        error = widgets.HTML(value=markdown(f'#Error received: {json["Error"]}'))
        display(error)
        return
    
    wh_json = get_warehouse_data(wid)
    wname = wh_json['Warehouse']['wname']
    
    df = pd.DataFrame(data=parts_info)
    df = df.style.set_caption(f'"{wname}" contains the following parts:')
    df = df.to_html(classes='center-table')
    with out:
        display(HTML(f'<div style="display: flex; justify-content: center;">{df}</div>'))
        # print(f'"{wname}" contains the following parts:')
        # for parts_tup  in parts_info:
        #     pname = parts_tup['pname']
        #     ptype = parts_tup['ptype']
        #     pid = parts_tup['pid']
        #     print(f'"{ptype} {pname}" (pid: {pid})')


In [157]:
def get_incoming_data():
    data = requests.get(base_url+'incoming')
    return data.json()

def get_outgoing_data():
    data = requests.get(base_url+'outgoing')
    return data.json()

def get_exchange_data():
    data = requests.get(base_url+'exchange')
    return data.json()

#print(get_incoming_data()) #'incoming'
#print(get_outgoing_data()) #'Outgoing'
#print(get_exchange_data()) #'exchange'


In [158]:
def add_type_column(row, type, wid):
    if type == "Exchange":
        row["type"] = "Exchange" + f" ({row['taction']})" 
    if type == "Incoming" or type == "Outgoing":
        row["type"] = type
    return row

def display_transac_in_warehouse(out, ware_id, u_id):
    wid = None
    uid = None
    try:
        wid = ware_id.new
        uid = u_id.new
    except:
        wid = ware_id
        uid = u_id
    
    out.clear_output()

    #validate user
    json = get_parts_in_warehouse(wid, uid)  
    parts_info = None
    try:
        parts_info = json['Parts_in_warehouse']
    except:
        error = widgets.HTML(value=markdown(f'#Error received: {json["Error"]}'))
        with out:
            display(error)
        return
    
    #get data
    incoming_data = get_incoming_data()['incoming']
    outgoing_data = get_outgoing_data()['Outgoing']
    exchange_data = get_exchange_data()['exchange']
    
    #filter by wid
    incoming_data = [row for row in incoming_data if row['wid'] == wid]
    outgoing_data = [row for row in outgoing_data if row['wid'] == wid]
    exchange_data = [row for row in exchange_data if row['wid'] == wid]

    # #add "type" column
    incoming_data = [add_type_column(row, "Incoming", wid) for row in incoming_data]
    outgoing_data = [add_type_column(row, "Outgoing", wid) for row in outgoing_data]
    exchange_data = [add_type_column(row, "Exchange", wid) for row in exchange_data]

    #create base dataframes
    incoming_df = pd.DataFrame(data=incoming_data)
    outgoing_df = pd.DataFrame(data=outgoing_data)
    exchange_df = pd.DataFrame(data=exchange_data)
    
    # with out:
    #     display(incoming_df)
    #     display(outgoing_df)
    #     display(exchange_df)

    #drop unnecessary data to have all tables equal to be able to join
    if incoming_data:
        incoming_df = incoming_df.drop(["incid", "sid"], axis=1)
    if outgoing_data:
        outgoing_df = outgoing_df.drop(["obuyer", "outid"], axis=1)
    if exchange_data:
        exchange_df = exchange_df.drop(["taction", "tranid"], axis=1)

    # with out:
    #     display(incoming_df)
    #     display(outgoing_df)
    #     display(exchange_df)
    
    
    master_df = pd.concat([incoming_df, outgoing_df, exchange_df], axis=0)
    if master_df.empty:
        error = widgets.HTML(value=markdown(f"#No transaction on warehouse {wid}"))
        with out:
            display(error)
        return
    master_df['tdate'] = pd.to_datetime(master_df['tdate'], format='%a, %d %b %Y %H:%M:%S GMT')
    master_df = master_df.sort_values(by='tdate', ascending=False)
    master_df['tdate'] = master_df['tdate'].dt.strftime('%d_%m_%Y %H:%M:%S')
    master_df.reset_index(inplace=True)
    master_df = master_df.drop(["index"], axis=1)
    master_df = master_df.style.set_caption(f"Transactions for Warehouse: {wid}")
    
    
    with out:
        display(master_df)
        

        

In [159]:
local_choices = ['profit', 'lowstock', 'material', 
                 'expensive', 'suppliers', 'leastcost'
                 , 'receivesmost']
global_choices = ['mostrack', 'mostincoming', 'mostdeliver',
                  'mosttransactions', 'leastoutgoing', 'mostcity']

part_info_choices = ['Part Prices', 'Parts Supplied by Supplier', 'Parts in a Warehouse']

main_choices = ['Local Statistics', 'Global Statistics', 'Transactions', 'Part Information']

#Add choices that need wid + uid here
choices_need_uid = ['Local Statistics', 'Transactions', 'Parts in a Warehouse']



main_selection = widgets.Dropdown(options = main_choices, value=main_choices[0])
entity_selection = widgets.Dropdown(options =  ['Warehouse','User'])
statistic_choice = widgets.Dropdown(
    options = local_choices)
choice_output = widgets.Output()

#Supplier selection for parts and Wid selection for parts and transactions
supplier_choice = widgets.BoundedIntText(value=1, description='Supplier:', min=1)
wid_choice = widgets.BoundedIntText(value=1, description='Wid:', min=1)

user_id = widgets.BoundedIntText(value=None, description='User ID:', min=1)
amount_displayed = widgets.BoundedIntText(value=None,description='Display #:', min=1)
display_all_btn = widgets.Button(description='Display All: Off')
    

graph_output = widgets.Output()

#display(main_selection, statistic_choice)
#reset widget values and return displays that may be 'default'
def reset_choices():
    #Dispaly wid and uid selection when needed
    if main_selection.value in choices_need_uid or statistic_choice.value in choices_need_uid: 
        user_id.layout.display = None
        wid_choice.layout.display = None
    else:
        user_id.layout.display = 'none'
        wid_choice.layout.display = 'None'

    #reset values
    supplier_choice.value = 1
    wid_choice.value = 1
    amount_displayed.value = 1
    amount_displayed.disabled = False
    display_all_btn.description = 'Display All: Off'
    

    #reset to default visibility
    display_all_btn.layout.display = 'none'
    supplier_choice.layout.display = 'none'
    amount_displayed.layout.display = 'none'
    statistic_choice.layout.display = None

def display_part_selections(change):
    user_id.layout.display = 'none'
    reset_choices()
    current = change.new
    if current == 'Parts Supplied by Supplier':
        supplier_choice.layout.display = None
    elif current == 'Parts in a Warehouse':
        wid_choice.layout.display = None
    elif current == 'Part Prices':
        display_all_btn.layout.display = None
        amount_displayed.layout.display = None

def update_statistic_choice(change):
    reset_choices()
    current = change.new
    new_choices = None
    if current == 'Local Statistics':
        new_choices = local_choices
    elif current == 'Global Statistics':
        new_choices = global_choices
    elif current == 'Part Information':
        new_choices = part_info_choices
    elif current == 'Transactions':
        statistic_choice.options = []
        statistic_choice.value = None
        statistic_choice.layout.display = 'none'
        #wid_choice.layout.display = None
        display_transac_in_warehouse(graph_output, wid_choice.value, user_id.value)
        return
    statistic_choice.options = new_choices
    statistic_choice.value = new_choices[0]
    
#button toggle
#note: b is a placeholder parameter, need it to work lol
def button_click(output, b):
    amount_displayed.disabled = not amount_displayed.disabled
    if display_all_btn.description == 'Display All: On':
        display_all_btn.description = 'Display All: Off'
        display_part_prices(output, amount_displayed.value)
    elif display_all_btn.description == 'Display All: Off':
        display_all_btn.description = 'Display All: On'   
        display_part_prices(output, None)  

    

reset_choices()
display_box = widgets.HBox([main_selection, statistic_choice])
# selection_box = widgets.GridBox([wid_choice, supplier_choice, user_id],
#                                layout=widgets.Layout(grid_template_columns="repeat(3, 300px)"))
selection_box =  widgets.HBox([wid_choice, supplier_choice, user_id, amount_displayed, display_all_btn])

# display_box = widgets.HBox([main_selection, statistic_choice, wid_choice, supplier_choice])

display_box.layout.justify_content = 'center'
selection_box.layout.justify_content = 'center'

centralized_box = widgets.VBox([display_box,selection_box])
centralized_box.layout.justify_content = 'center'
display(centralized_box)

# display(display_box)
# display(selection_box)

#sets main_selection callback to update the choice of statistics
main_selection.observe(update_statistic_choice, names='value')
statistic_choice.observe(display_part_selections, names='value')



    

In [160]:
# JavaScript code for displaying and hiding the help message
help_script = """
function toggleHelp() {
  var helpDiv = document.getElementById('help-message');
  helpDiv.style.display = (helpDiv.style.display === 'none' || helpDiv.style.display === '') ? 'block' : 'none';
}
"""

help_message ="""# Help
Use the accordion above to view each endpoint and what it does.
Different statistic types need different information to work.
### Legend
- **Wid**: Warehouse ID. Indicates the warehouse to view.
- **User ID**: User ID used to access specific warehouse information. Only required in **local** statistics, and other specs that involve **warehouse** information . User must have access to warehouse to view.
- **Display #**: Amount to display.
- **Supplier**: Supplier ID used to select a specific supplier.

The graph output will show below. Be sure to have proper fields set, and read error messages otherwise to fix any issues.
"""

# Button and help message HTML
help_button_html = """
<button onclick="toggleHelp()">Show Help</button>
<div id="help-message" style="display:none; position: fixed; top: 50%; left: 50%; transform: translate(-50%, -50%); padding: 20px; background-color: #f9f9f9; border: 1px solid #ccc; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);">
  <div> {} </div>
  <button onclick="toggleHelp()">Close</button>
</div>
""".format(markdown(help_message))

# Display the button and help message
display(HTML('<script>{}</script>'.format(help_script)))
display(HTML(help_button_html))

In [161]:
#graph_output = widgets.Output()

def view_local_statistic(change):
    graph_output.clear_output()
    current_stat = statistic_choice.value
    new_val = change.new
    func = None

    main_stat = main_selection.value

    with graph_output:
        if current_stat == 'profit':
            func = display_warehouse_profit
        elif current_stat == 'lowstock':
            func = display_warehouse_lowstock
        elif current_stat == 'material':
            func = display_warehouse_material
        elif current_stat == 'expensive':
            func = display_warehouse_expensive
        elif current_stat == 'suppliers':
            func = display_warehouse_suppliers
        elif current_stat == 'leastcost':
            func = display_warehouse_leastcost
        elif current_stat == 'receivesmost':
            func = display_warehouse_receivesmost
        
        #ik these two are a bit messy, but it's lowkey convenient
        elif current_stat == 'Parts in a Warehouse': #part of part info
            func = display_parts_in_warehouse
        elif main_stat == 'Transactions':
            # display_transac_in_warehouse(graph_output, wid_choice.value)
            # return
            func = display_transac_in_warehouse
        if func is None:
            print('ERROR: Invalid option')
        if change.owner.description.startswith('User'):
            func(graph_output, wid_choice.value, new_val)
        else:
            func(graph_output, new_val, user_id.value)
    pass   

def view_statistic(option):
    graph_output.clear_output()
    curr = option.new
    with graph_output:
        if curr == 'mostrack':
            display_warehouse_with_most_racks()
        elif curr == 'mostincoming':
            display_warehouse_with_most_incoming_transactions()
        elif curr=='mostdeliver':
            display_warehouse_with_most_exchanges()
        elif curr == 'mosttransactions':
            display_top_three_users_with_most_transactions()
        elif curr == 'leastoutgoing':
            display_warehouse_with_least_outgoing()
        elif curr == 'mostcity':
            display_cities_with_most_warehouse_transactions()
        elif curr == 'Part Prices':
            display_part_prices(graph_output, amount_displayed.value)
        elif curr == 'Parts Supplied by Supplier':
            display_supplied_parts(graph_output, supplier_choice.value)
        elif curr == 'Parts in a Warehouse':
            display_parts_in_warehouse(graph_output, wid_choice.value, user_id.value)
    pass   
    
statistic_choice.observe(view_statistic, names = 'value')

wid_choice.observe(view_local_statistic, names = 'value')
user_id.observe(view_local_statistic, names = 'value')


#For Part Information
display_all_btn.on_click(functools.partial(button_click, graph_output))

#ik the if statements are redundant but 'observe' really doesn't like nested ifs
# and the if statement was already pretty long lol
if display_all_btn.description == 'Display All: Off':
    graph_output.clear_output()
    amount_displayed.observe(functools.partial(display_part_prices, graph_output), names='value')
if display_all_btn.description == 'Display All: Off':
    supplier_choice.observe(functools.partial(display_supplied_parts, graph_output), names='value')
#--------------------

display(graph_output)

Output()